## $\color{red}{\text{Conversion of sampling frequency and STFT}}$

#### Maël Le Guillouzic
*TP de TSIA 201 n°1*

In [1]:
# On réalise les imports nécessaires pour le bon fonctionnement du notebook
import os, sys, wave, struct
import sys
import numpy as np
import pyaudio
from IPython.display import Audio
import matplotlib.pyplot as plt
from scipy.io.wavfile import write

print("Python version")
print(sys.version)

Python version
3.11.4 (tags/v3.11.4:d2340ef, Jun  7 2023, 05:45:37) [MSC v.1934 64 bit (AMD64)]


## Introduction
 Les cellules suivantes sont fournies par le template. Elles permettent d'ouvrir un fichier audio, de le lire ainsi que d'en obtenir les principales caractéristiques et enfin de l'afficher.

In [2]:
def load_sound(file):
    return wave.open(file, 'rb')

def play_sound(file, chunk = 1024):
    wf = load_sound(file)
    p = pyaudio.PyAudio()
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True)
    data = wf.readframes(chunk)

    while data:
        stream.write(data)
        data = wf.readframes(chunk )

    stream.stop_stream()
    stream.close()
    p.terminate()
    
    
def plot_sound(data, times, name='default_name', save=False):
    plt.figure(figsize=(30, 4))
    plt.fill_between(times, data)
    plt.xlim(times[0], times[-1])
    plt.xlabel('time (s)')
    plt.ylabel('amplitude')
    # plt.ylim(-15000, 15000)
    if save:
        plt.savefig(name+'.png', dpi=100)
    plt.show()

### Reading and playing .wav file

Ici on choisit le son qu'on utilisera pour le notebook.

In [130]:
data_path = os.getcwd()
music = 'caravan_48khz.wav'
sound = os.path.join(data_path, music) 

On peut écouter ce son choisit grace au module Audio de `Ipython.display`

In [ ]:
wavefile = load_sound(sound)
print(wavefile.getparams())

play = False
if play :
    play_sound(sound)
wavefile.close()

Audio(sound)

Pour étudier cet audio, on va devoir le convertir en *np.array*. C'est l'opération ici réalisée.
On en profite pour afficher sa caractéristique principale sur laquelle on va travailler, sa fréquence d'échantillonage : **48 kHz**

In [ ]:
wavefile = wave.open(sound, 'r')
Fs = int(wavefile.getframerate())
num_samples = int(wavefile.getnframes())
print(f"Frequence d'échantillonnage: {Fs/1000} kHz et nombre d'échantillons: {num_samples}")

data = wavefile.readframes(num_samples)
data = struct.unpack('{n}h'.format(n=num_samples), data)
x = np.array(data)

print('Durée du son: ', num_samples/Fs, 's')

In [ ]:
timestep = 1/float(Fs)
times = np.arange(len(x))*timestep
plot_sound(x, times)

## Partie 1 : Conversion of Sampling Rate

### Question 1
*On souhaite convertir cet audio de la fréquence d'échantillonage 48kHz à celle 32kHz. Quelle chaine va t'on utiliser pour cela ?*

L'opération souhaitée est un resampling de facteur **2/3**.
On peut donc mettre en place la chaine suivante, où on a dans l'ordre :
- **Une insertion de zéros** d'ordre 2
- Un **filtrage** par un filtre passe-bas de fonction de transfert $H(e^{2i\pi\nu}) = L \;\; \text{si} \;\; \forall \nu < \frac{1}{6}$ et $H(e^{2i\pi\nu}) = 0 \;\; \text{si} \;\; \frac{1}{6} < \nu < \frac{1}{2}$
- Une **décimation** d'ordre 3


<img src="chaine.jpg" width="500" />

### Question 2

On va donc générer un filtre de réponse impulsionelle $h(n)$ qui doit respecter le critère suivant :

- Ecart d'au moins 50 dB entre la bande passante et la bande non-passante
- Une bande de transition relativement faible

On va pour cela choisir un filtre d'odre très grand, qui respectera ainsi facilement les contraintes.

In [ ]:
from scipy.signal import remez, lfilter, freqz

# Fréquence d'échantillonnage
Fs = 48000

# M et L définissent notre facteur de resampling
M=3
L=2

# Fc est la fréquence de coupure
Fc = np.min([Fs/(2*M), Fs/(2*L)])
Fc_norm = Fc/Fs # on normalise
Fcp_norm = (Fc+0.05*Fc)/Fs # Fcp correspond à la fréquence de début de la bande non-passante
# Fcp_norm - Fc_norm = Largeur de la bande de transition
# print(Fc_norm)

N = 500
h = remez(N,[0,Fc_norm,Fcp_norm,0.5],[L,0])
# print(h)

nu,H = freqz(h)
plt.figure(figsize=(15,5))
plt.plot(np.linspace(0,0.5,len(H)),20*np.log10(np.abs(H)))
plt.title('Frequency response of the low-pass filter')
plt.grid()
plt.show()

Pour construire notre chaine de resampling, on va tout de suite définir les trois opérations importantes, qui seront ré-utilisées dans les questions tout au long de la partie 1 : 
- Insertion de zéros
- Filtrage
- Déciimation

In [135]:
# Fonction d'insertion de zéros
def insert_zeros(x,L):
    temp = np.zeros(len(x)*L)
    temp[::L] = x
    return temp

# Fonction de filtrage (filtre par notre filtre passe-bas déterminé plus haut)
def filter(x,h):
    return lfilter(h,1,x)

# Fonction de resampling
def decimation(x,L):
    return x[::L]

On va ainsi pouvoir effectuer le resampling

In [ ]:
# opération 1 : insertion de zéros
upsampled = insert_zeros(x,L)
# opération 2 : filtrage
filtered = lfilter(h,1.0,upsampled)
# opération 3 : decimation
final = decimation(filtered,M)

# Vérifications
print(f"Verif rapide {len(filtered)}={M*len(final)}")
print(f"len final {len(final)}, len x {len(x)}, ce qui donne un facteur de 2/3 : c'est ce qu'on voulait !")

# On peut maintenant afficher le signal resamplé
step = 1/(Fs*L/M)
times = np.arange(len(final))*step
plot_sound(final, times)

# On peut maintenant sauvegarder le signal resamplé
write('caravan_32khz.wav',int(Fs*L/M),final.astype(np.int16))

La taille du vecteur obtenue en sortie est bien inférieure d'un facteur 2/3 par rapport au signal d'origine. Au nouvel échantillonage, la durée est toujours de 16 secondes. Tout semble indiquer que l'opération a réussi. Ecoutons pour en être sur :

In [ ]:
# Audio(final,rate=int(Fs*(L/M)))
write('caravan_32khz.wav',int(Fs*L/M),np.array(final,dtype=np.int16))
Audio('caravan_32khz.wav')

On retrouve notre musique de départ !

Ce qui est pratique avec la manière dont on a codé cette partie, c'est qu'on peut choisir **M** et **L** de manière à obtenir le facteur de resampling de notre choix. Essayons donc avec $L=1$ et $M=12$ par exemple (on essaie de conserver M et L premiers entre eux). On va alors entendre à l'oreille que les aigues disparaissent, ce qui est logique.

Pour cela on va juste intégrer notre code précédent dans une fonction (cela nous sera aussi utile pour comparer les temps d'éxecution)

In [138]:
def resample(x,L,M):
    Fs = 48000
    Fc = np.min([Fs/(2*M), Fs/(2*L)])
    Fc_norm = Fc/Fs
    Fcp_norm = (Fc+0.05*Fc)/Fs
    N = 500
    h = remez(N,[0,Fc_norm,Fcp_norm,0.5],[L,0])
    upsampled = insert_zeros(x,L)
    filtered = filter(upsampled,h)
    final = decimation(filtered,M)
    return final

In [ ]:
# Resample avec L=1 et M=12
final12 = resample(x,1,12)
Audio(final12,rate=int(Fs/12))

Vérifions une dernière fois que notre audio sauvegardé l'est bien à la bonne fréquence.

In [ ]:
wavefile = wave.open('caravan_32khz.wav', 'r')
Fs = int(wavefile.getframerate())
num_samples = int(wavefile.getnframes())
print(f"Frequence d'échantillonnage: {Fs/1000} kHz et nombre d'échantillons: {num_samples}")
print(f"Durée du son: {num_samples/Fs} s")

In [ ]:
wavefile = wave.open(sound, 'r')
x = np.array(struct.unpack('{n}h'.format(n=int(wavefile.getnframes())), wavefile.readframes(int(wavefile.getnframes()))))
Fs = wavefile.getframerate()
wavefile.close()

y = resample(x,2,3)

step = 1/Fs
times = np.arange(len(x))*step
plot_sound(x, times)

step = 1/(Fs*(2/3))
times = np.arange(len(y))*step
plot_sound(y, times)

### Question 4

On peut décomposer grâce aux composantes poly-phase cette opération de resample afin de gagner en efficacité. Pour cela on peut commencer par vérifier l'équivalence demandée. D'une part (avec $y'$ le signal situé juste après le bloc $z^{-1}$):
$$ Y'(z) = X(z^2) z^{-1} $$
$$ Y(z) = \frac{1}{3} \sum_{k = 0}^{2} z^{-\frac{1}{3}} X\left(z^{\frac{2}{3}} e^{-2i\pi \frac{k}{3}}\right)$$
D'autre part (avec $y'$ le signal situé juste après le bloc de décimation par 3) :
$$ Y'(z) = \frac{1}{3} \sum_{k = 0}^{2} z^{\frac{1}{3}} X\left(z^{\frac{2}{3}} e^{-2i\pi \frac{k}{3}}\right) $$
$$ Y(z) = \frac{z^{-1}}{3} \sum_{k = 0}^{2} z^{\frac{2}{3}} X\left(z^{\frac{2}{3}} e^{-2i\pi \frac{k}{3}}\right) = \frac{1}{3} \sum_{k = 0}^{2} z^{-\frac{1}{3}} X\left(z^{\frac{2}{3}} e^{-2i\pi \frac{k}{3}}\right) $$
On a donc bien égalité des deux schéma bloc.

On va utiliser cette équivalence pour trouver un schéma équivalent au resampling. On pourra alors 

<img src="resampling_polyphase.jpg" alt="Schéma équivalent en utilisant la méthode des composantes polyphases" width="500"/> 

### Question 5 
Programmons notre implémentation polyphase. Pour cela on va commencer par définir deux opérations qui nous serons utiles : le décalage à droite et à gauche d'un montant voulu.

In [142]:
# On définit des fonctions décalage à gauche et à droite
def shift_right(x,n):
    return np.pad(x[n:],(0,n),mode='constant')

def shift_left(x,n):
    return np.pad(x[:len(x)-n],(n,0),mode='constant')

# test = np.array([1,2,3,4,5,6])
# print(shift_right(test,2))
# print(shift_left(test,2))

Puis on peut définir notre fonction principale. **Attention** cette implémentation polyphase ne marche que pour un facteur de resampling de 2/3.

In [143]:
# Programmons cette implémentation polyphase de la fonction de resampling 
# pour un facteur de resampling de 2/3

def resample_poly_2_3(x,L=2,M=3):
    # On réalise les shifts en entrée
    x0,x1,x2,x3,x4,x5 = x,shift_right(x,1),shift_left(x,1),x,shift_left(x,1),shift_left(x,2)
    # Puis on fait les décimations
    x0,x1,x2,x3,x4,x5 = map(lambda x:decimation(x,M),[x0, x1, x2, x3, x4, x5])
    # Et ici on réalise les polyphases
    x0, x1, x2, x3, x4, x5 = x0[::3], x1[1::3], x2[2::3], x3[::3], x4[1::3], x5[2::3]

    # On somme les polyphases de chaque branche
    x01 = np.sum([x0,x1,x2],axis=0)
    x02 = np.sum([x3,x4,x5],axis=0)

    # On insère des zéros
    x01 = insert_zeros(x01,L)
    x02 = insert_zeros(x02,L)

    # On somme les deux branches
    x01=shift_left(x01,1)
    y = np.sum([x01,x02],axis=0)
    return y

On va recharger le son depuis le départ pour éviter toute mauvaise suprise. Puis on va pouvoir executer notre fonction de resample polyphase et vérifier qu'elle fonctionne correctement.

Par habitude, je sépare la cellule de définition de ma fonction de celle d'éxécution.

In [144]:
wavefile = wave.open(sound,'r')
x = np.array(struct.unpack('{n}h'.format(n=int(wavefile.getnframes())), wavefile.readframes(int(wavefile.getnframes()))))
Fs = wavefile.getframerate()
wavefile.close()

In [ ]:
y_poly = resample_poly_2_3(x=x)
# print(f"len y_poly {len(y_poly)}, len x {len(x)}")

step = 3/(Fs*2/3)
times_poly = np.arange(len(y_poly))*step
plot_sound(y_poly,times_poly)

Audio(y_poly,rate=int(Fs*(L/(3*M))))

#### Question 6

On cherche à évaluer l'amélioration avec notre méthode polyphase.
Grace au module `time`, on peut afficher simplement le temps d'éxecution de chaque fonction.

In [ ]:
import time

start = time.time()
y = resample(x,2,3)
end = time.time()

start_poly = time.time()
y_poly = resample_poly_2_3(x,2,3)
end_poly = time.time()

tps1 = end-start
tps2 = end_poly-start_poly

if tps2==0:
    tps2 = 0.0001

print(f"Temps d'exécution du resample classique: {tps1}")
print(f"Temps d'exécution du resample poly: {tps2}")
print(f"Amélioration d'un facteur {tps1/tps2}")

On remarque que le temps d'éxecution varie grandement selon les runs.
Pour avoir une meilleur idée de ce dernier, éxecutons donc chaque fonction 100 fois, et en réalisant la moyenne des temps d'éxecution on aura une bonne idée du facteur d'amélioration moyen.

In [ ]:
# Puisque d'itérer 100 fois chaque fonction prend du temps,
# on définit le paramètre q6 qui nous permet d'éxecuter ou non cette partie
q6 = True

# Calculons l'amélioration du temps d'exécution moyenne
if q6:
    n = 100
    tps1,tps2 = [],[]
    for i in range(n):
        start = time.time()
        y = resample(x,2,3)
        end = time.time()
        tps1.append(end-start)
        
        start_poly = time.time()
        y_poly = resample_poly_2_3(x,2,3)
        end_poly = time.time()
        tps2.append(end_poly-start_poly)

    print(f"Temps d'exécution moyen du resample classique: {np.mean(tps1)}")
    print(f"Temps d'exécution moyen du resample poly: {np.mean(tps2)}")
    print(f"Amélioration moyenne d'un facteur {np.mean(tps1)/np.mean(tps2)} en moyenne")

On améliore en moyenne d'un facteur 12 : **c'est bien !**

## Partie 2 : STFT

*Dans cette partie on va chercher à implémenter et étudier la STFT, qui nous permettra d'observer le spectrogramme de notre signal*

#### Question 1
Prenons une fenetre de Hann de longueur $N_w$, calculons et affichons sa DFT. On essaiera aussi d'obtenir la largeur du lobe principal. On choisit pour cela un M *très grand* afin de réaliser une opération de zéro padding et d'augmenter la résolution de la DFT.

In [148]:
N = x.shape[0] # longueur du signal
Nw = 512 # Taille de la fenetre d'analyse
M = 4*4096 # Ordre de la tfd
L = M/2+1

In [ ]:
# Prenons une fenêtre de Hanning
w = np.hanning(Nw)

# On réalise la TFD, si M>Nw, on réalise un zero padding
W = np.fft.fft(w,M)
Wlog = 20*np.log10(np.abs(W))
freqs = np.fft.fftfreq(M)

# Affichons cette TFD
plt.figure(figsize=(15,3))
plt.subplot(122)
plt.plot(freqs,Wlog)
plt.title('TFD de la fenêtre de Hanning')
plt.xlabel('Fréquence')
plt.xlim(0,0.04)
plt.ylim(-75, 50)

plt.subplot(121)
plt.plot(w)
plt.title('Fenêtre de Hanning')
plt.show()

Pour rechercher la largeur du lobe principal, on va définir une petite fonction qui va avancer le long du vecteur obtenue par la DFT, jusqu'a ce que le sens de variation de ce vecteur change.

In [ ]:
Wrecherche = np.fft.fft(w,M)
imax = np.argmax(Wlog)
print(f"Fréquence maximale: {imax} Hz")

# En partant du max, on va avancer jusqu'à ce que la TFD change de sens de variation
def largeur_bande(Wlog):
    recherche = True
    ilarge = imax
    while recherche:
        if Wlog[ilarge] > Wlog[ilarge+1]:
            ilarge +=1
        else:
            recherche = False
    return Wlog[ilarge],ilarge

largeur,ilarge = np.abs(largeur_bande(Wlog))
print(f"Largeur du lobe principal: {2*largeur} Hz")


#### Question 2
*2. Note that the expression (1), taken at fixed λ, can be written as a convolution and deduce an inter-
pretation of the STFT in terms of filtering. Explain the role of the corresponding filter (low-pass?
band-pass? high-pass?). As a linear phase FIR filter, specify its type according to its length (even or odd).*

Quand on observe l'équation (1) :
$$W_x(\lambda, b) = \sum_{n \in \mathbb{Z}} x(n) w(n - b) e^{-j 2 \pi \lambda n} \tag{1}$$
on peut la ré-écrire sous forme de la convolution suivante :
$$
W_x(\lambda, b) = \left[ h_k * x \right](b) \:\: , \:\: h_k = \left( w(n) e^{-j 2 \pi \lambda n} \right) \tag{2}$$

On remarque alors que :
- Si $ \lambda = 0$ alors on ne filtre que par la fenetre de Hanning. Or après avoir tracé sa TFD, on voit que Hanning est **symétrique** et **centrée sur 0**. De plus, elle agit comme un filtre *passe-bande* autour de son lobe principal situé en 0.
Ainsi, si $\lambda = 0$ c'est un filtre **passe-bas**.
- Si $ \lambda \neq 0$, alors cela revient à décaler la fenetre de Hanning sur une autre fréquence. On effectue donc un **passe-bande** autour de la fréquence $\lambda$

De plus, la fenetre de Hanning étant symétrique, les types 3 et 4 sont d'office éliminés. On peut alors déterminer le type grace à la longueur $N_w$ du filtre:
- Si $N_w$ est pair, alors le filtre est symétrique pair, c'est un filtre FIR de type 2.
- Si $N_w$ est impair, alors le filtre est symétrique impair, c'est un filtre FIR de type 1.

#### Question 3

Cette nouvelle forme se devine directement de l'écriture (2), par symétrie de la convolution.
Elle revient non pas à décaler la fenêtre le long du signal traité; mais à fixer la fenêtre et faire *"défiler"* le signal à l'intérieur.
C'est cette nouvelle version qui est implémentée dans le template fourni pour le TP.

On a alors le lien entre les deux formes :
$$

\tilde{X}(\lambda, b) = \sum_{n \in \mathbb{Z}} x(n + b) w(n) e^{-2 i \pi \lambda n}
= \sum_{k \in \mathbb{Z}} x(k) w(k-b) e^{-2 i \pi \lambda (k-b)}
= e^{2 i \pi \lambda b} W_x(\lambda, b)
$$

In [151]:
"""
Cellule fournie dans le template du TP
"""
affich = 1 ; # affichage ou non du spectrogramme
R = Nw/4 # incrément sur les temps d'analyse, appelé hop size, t_a=uR
# Si R=Nw/2, on a un recouvrement de 50% entre deux fenêtres, etc ... Ici on a un recouvrement de 75%
Nt = (np.rint((N-Nw)/R)).astype(int) # nombre de trames

if affich:
    Xtilde = np.zeros((M,Nt),dtype=complex)
    for u in np.arange(0,Nt).reshape(-1): # boucle sur les trames
        deb = u*R+1 # début de trame
        fin = deb + Nw # fin de trame
        tx = np.multiply(x[np.arange(deb.astype(int),fin.astype(int))],w) # calcul de la trame 
        X = np.fft.fft(tx,M) # tfd à l'instant b
        if affich:
            Xtilde[:,u] = X
        Y = X.copy

def extents(f):
    delta = f[1]-f[0]
    return [f[0]-delta/2,f[-1]+delta/2]

#### Question 4

A partir du template fourni, on va définir la fonction `X_tilde` qui calcule directement le spectrogramme. On va ensuite calculer ce spectrogramme pour notre signal x, et en extraire une portion de fréquence.

In [152]:
def X_tilde(x,M,R):
    N = x.shape[0]
    Nw = 512
    w = np.hanning(Nw)
    Nt = int(np.rint((N-Nw)/R))
    Xtilde = np.zeros((M,Nt),dtype=complex)
    for u in np.arange(0,Nt).reshape(-1):
        deb = u*R+1
        fin = deb + Nw
        tx = np.multiply(x[np.arange(deb.astype(int),fin.astype(int))],w)
        X = np.fft.fft(tx,M)
        Xtilde[:,u] = X
    return Xtilde

In [153]:
# L'énoncé nous demande de calculer Xtilde pour M=32 et R=1
M = 32
R = 1
k = 3
Xtilde2 = X_tilde(x,M,R)

In [ ]:
print(Xtilde.shape)
x_k = Xtilde2[1,:]
print(f"x_k est de type : {x_k.dtype}")
print(f"x_k = {x_k}")

On obtient $x_k$ qui est complexe. En terme de filtrage, la STFT correspond à une série de filtrages passe-bande centrés sur différentes fréquences.
Quand on écoute $Re(x_k)$ on entend exactement notre musique de départ. Cela est du au fait que $M=32$ est très faible devant la taille du spectre audible. Ainsi pour un seul échantillon de la DFT, on couvre une large portion de fréquences. C'est pourquoi qu'on on écoute x_k on peut avoir l'impression de retrouver notre signal de départ.

In [ ]:
Re_xk = np.real(x_k)
Audio(Re_xk,rate=Fs)

In [ ]:
plt.figure(figsize=(15,3))
plt.subplot(121)
plt.imshow(
    X=20*np.log10(np.abs(Xtilde[0:int(L),:])), 
    aspect='auto',
    interpolation='none',
    origin='lower', 
    extent=[0,Nt*R/Fs+Nw/2,0,Fs/2])

plt.subplot(122)
plt.imshow(
    X=20*np.log10(np.abs(Xtilde2[:int(L/2),:])),
    aspect='auto',
    interpolation='none',
    origin='lower',
    extent=[0,Nt*R/Fs+Nw/2,0,Fs/2])

**Remarque 1 :** Si on prend $4*4096$ on observera le même spectrogramme à gauche et à droite.

**Remarque 2 :** On a en effet à droite, pour chaque instant temporelle, une information en fréquence qui est brouilée. C'est la présence de très nombreuses fréquences différentes qui rend impossible l'isolation d'une d'entre elles en particulier.

In [ ]:
from scipy.signal import spectrogram

Sxx = spectrogram(x, fs=48000)[2]
l=len(Sxx)
Sxx=np.concatenate([Sxx[l//2:],Sxx[:l//2]])

plt.figure(figsize=(15,3))
plt.plot(np.arange(-l//2,l//2)/l,Sxx)
plt.title('spectrum of the original sound')
plt.xlabel('frequencies')
plt.xlim(-0.02,0.2)

#### Question 5

On montre la condition demandée avec : 

<img src="condition_ola.jpg" alt="Démonstration de f(n)=1" width="600"/>

In [158]:
def ola(w = None,hop = None,Nb = 10): 
# function output = ola(w,hop,Nb)
# realise l'addition-recouvrement de la fenetre w,
# avec un décalage hop et un nombre Nb de fenetres.
# par defaut Nb = 10;
    w = w[:,np.newaxis]
    N = len(w)
    output = np.zeros(((Nb-1)*hop+N,1)) # réserve l'espace memoire
    for k in np.arange(0,Nb).reshape(-1):
        deb = k* hop
        fin = deb+N
        output[np.arange(deb,fin)] = output[np.arange(deb,fin)] + w # OLA
    return output

In [ ]:
# Partie 1
N = 512
Nb = 10
hop = N//4
Nt = (np.rint((N-M)/R)).astype(int)

# On définit la fenêtre de Hanning
h = np.hanning(N)
wp = h**2

h_ola = ola(wp,hop,Nt)

# On calcule le facteur de normalisation
norm = np.max(h_ola)
h_ola = h_ola/norm

plt.figure(figsize=(20,3))
plt.subplot(131)
plt.plot(h)
plt.title('Fenêtre de Hanning')

plt.subplot(132)
plt.plot(h_ola)
plt.title('OLA de la fenêtre de Hanning')
plt.axhline(y=1, color='r', linestyle='--')
# plt.axvline(x=0.20*len(h_ola), color='r', linestyle='--')
# plt.axvline(x=0.80*len(h_ola), color='r', linestyle='--')

# plt.subplot(133)
# plt.plot(np.arange(0,500),h_ola[0:500], label="0-500")
# plt.plot(np.arange(500,1000),h_ola[len(h_ola)-500:len(h_ola)], label="2750-3250")

Une fois normalisé par le bon facteur, **on a bien $f(n)=1$** sur la totalité de l'intervalle !
La condition est donc parfaitement respectée et la fenetre de Hanning permet une resconctruction parfaite !

### Question 6
On va implémenter la reconstruction avec OLA. Pour cela nous utiliserons une fenetre de Hanning, puisqu'elle vérifie la condition de resconstruction parfaite démontrée à la question 5.

In [160]:
M = 32
Nw = M
R = 1
XtildeOLA = X_tilde(x,M,R)
y = np.zeros(len(x))

In [161]:
# On rapelle qu'on cherche à reconstruire x à partir de XtildeOLA qui est la STFT de x (son spectrogramme donc)
# On va donc pour chaque échantillon de XtildeOLA:
# - Calculer la TFD inverse
# - Multiplier par une fenêtre de Hanning
# - Réaliser l'OLA de ces fenêtres
def reconstruction(XtildeOLA):
    y = np.zeros(M+R*XtildeOLA.shape[1])
    for i in range(len(XtildeOLA[0])):
        x_partiel = np.real(np.fft.ifft(XtildeOLA[:,i],M))
        # print(x_partiel.shape)
        w = np.concatenate([np.hanning(M),np.zeros(np.abs(M-Nw))])
        x_w_partiel = x_partiel*w
        y[i*R:i*R+Nw] += x_w_partiel
    return y

y = reconstruction(XtildeOLA)
y = np.concatenate([y,np.zeros(len(x)-len(y))])

# Normalisons
y = y/np.max(y)

In [ ]:
Audio(y,rate=Fs)

Nous observons la musique d'origine. La reconstruction OLa fonctionne fonctionne de manière tout à fait correcte. Observons tout de même son allure et calculons l'erreur quadratique moyenne.

In [ ]:
diff = np.abs((x/np.max(x))-(y/np.max(y)))
t = np.arange(len(x))/Fs

fig = plt.figure(figsize=(15,3))
plt.subplot(211)
plt.plot(t,x)
plt.title('Signal reconstruit')
plt.subplot(212)
plt.plot(t,diff**2)
plt.title('Différence entre signal original et reconstruit')
plt.tight_layout()

In [ ]:
error = np.mean(diff**2)
print(f"Erreur quadratique moyenne de reconstruction: {error}")

if error < 1e-3 :   
    print("Reconstruction parfaite validée!")
else:
    print("La reconstruction n'est pas parfaite")

On a définit une erreur quadratique moyenne acceptable comme étant inférieur à 1/1000. On obtient bien quelque chose de très proche !

Notre reconstruction nous permet donc de retrouver le signal d'origine !

### Question 7

On veut implémenter un equalizer. Cela correspond à choisir un vecteur de $w_k$ qui nous permettra d'augmenter l'effet de certaines fréquences et de diminuer l'effet d'autres.
Pour cela, comme l'indique l'énoncé, on va répeter exactement ce qu'on a fait avant, en ajoutant juste une multiplication par un vecteur $w_k$ juste avant d'effectuer la TFD inverse.

$w_k$ sera définit grace à 3 curseurs, chacun augmente un tier du vecteur :
- t1 augmente le premier tier et correspond donc à une augmentation des basses fréquences
- t2 augmente le second tier
- t3 augmente le troisième tier

In [165]:
def equalizer(XtildeOLA,w_k):
    y = np.zeros(M+R*XtildeOLA.shape[1])
    for i in range(len(XtildeOLA[1])):
        x_partiel = np.real(np.fft.ifft(XtildeOLA[:,i]*w_k,M))
        # print(x_partiel.shape)
        w = np.concatenate([np.hanning(M),np.zeros(M-Nw)])
        x_w_partiel = x_partiel*w
        y[i*R:i*R+Nw] += x_w_partiel
    return y

Définissons notre vecteur $w_k$ :

In [ ]:
import ipywidgets as widgets

w_k = np.ones(XtildeOLA.shape[0])

def def_w_k(t1,t2,t3):
    t = M//3
    w_k[:t] *= t1
    w_k[t:2*t] *= t2
    w_k[2*t:] *= t3
    print(w_k)

t1 = widgets.FloatSlider(value=1,min=0,max=12,step=0.1,description='t1')
t2 = widgets.FloatSlider(value=1,min=0,max=12,step=0.1,description='t2')
t3 = widgets.FloatSlider(value=1,min=0,max=12,step=0.1,description='t3')

ui = widgets.HBox([t1,t2,t3])
out = widgets.interactive_output(def_w_k, {'t1':t1,'t2':t2,'t3':t3})
display(ui,out)

Et vérifions que notre méthode fonctionne bien.

In [ ]:
y_equalized = equalizer(XtildeOLA,w_k)
Audio(y_equalized,rate=Fs)

**Limitation :** Comme on l'avait vu à une question précédente, le decoupage de la plage fréquentielle en 32 ne permet pas d'isoler correctement les plages de fréquences *audibles*.
En effet l'audition fonctionne en échelle logarithmique, et il faudrait donc ajuster $w_k$ non pas selon 3 tiers, mais selon 3 plages égales au niveau algorithmique.
Cela permetrait d'obtenir un equalizer bien meilleur niveau audition.






#### $\color{white}{\textit{Fin du TP de Maël Le Guillouzic}}$